In [1]:
import pandas as pd
import numpy as np
import getschools as gs

In [25]:
def get_df():
    
    ret = pd.read_excel("../raw/CCJEF trial excel teachers.xlsx",
                        sheetname="Sheet4",skiprows=1)
    
    ret = ret[ret["District Name"].str.contains("Total")]
    
    ret = gs.clean_colnames(ret)
    
    ret["district"] = ret["district_name"].apply(lambda x: x.strip().replace(" Total",""))
    
    ret = gs.add_district_matches(ret)
                                             
    ret["district_code"] = ret["dist_code"]
    
    ret = ret[["district_code","district","grand_total"]]
    
    ret = gs.add_drg(ret)
    
    return ret

get_df().sort_values(by="grand_total",ascending=False).to_csv("../clean/for_db/teacher_compensation_ccjef.csv",index=False)

,district_code,district,grand_total,drg
62,0570011,Greenwich School District,86721.74316,B
107,1030011,Norwalk School District,85443.38986,H
123,1170011,Redding School District,84491.21951,A
187,1610011,Wilton School District,84140.54542,A
89,0850011,Monroe School District,81538.09005,B
183,1570011,Weston School District,81453.70061,A
6,0040011,Avon School District,80123.66197,B
130,2090012,Regional School District 09,79759.33333,A
94,0900011,New Canaan School District,79662.70323,A
171,None,Unified School District #1,79165.64356,None


In [26]:
get_df()[get_df()["district"].str.contains("State")]

,district_code,district,grand_total,drg
196,None,State,68786.73711,None


In [27]:
get_df()["grand_total"].median()


65121.3125

In [28]:
get_df()["grand_total"].mean()

64989.412536395932

In [33]:
len(get_df()[get_df()["district_code"].isnull()])

15

In [34]:
len(get_df()[get_df()["district_code"].notnull()])

182

In [42]:
def drg_report():
    df = get_df()
    
    df = df.groupby("drg").agg({"grand_total":np.median}).reset_index()
    
    return df

drg_report()#.to_csv("../clean/for_db/drg_teacher_compensation_ccjef",index=False)

,drg,grand_total
0,A,79662.703230
1,B,71772.560280
2,C,64867.747545
3,D,67826.620525
4,E,63440.823205
5,F,63686.005680
6,G,64659.694740
7,H,70578.395200
8,I,61782.333330


In [43]:
def state_report():
    df = get_df()
    df = df[df["district"] == "State"]
    
    return df

state_report()#.to_csv("../clean/for_db/state_teacher_compensation_ccjef",index=False)

,district_code,district,grand_total,drg
196,None,State,68786.73711,None


In [51]:
state_grand_total = float(state_report()["grand_total"])
state_grand_total

68786.73711

In [47]:
drg_df = drg_report()
state_df = state_report()



In [53]:
def get_drg_total(drg):    
    if drg is None: 
        return
    
    return float(drg_df[drg_df["drg"] == drg]["grand_total"])

get_drg_total("A")

79662.70323

In [58]:
def combined_df():
    df = get_df()
    
    df["drg_grand_total"] = df["drg"].apply(get_drg_total)
    df["state_grand_total"] = state_grand_total
    df["school_code"] = df["district_code"]
    return df

combined = combined_df()
combined

,district_code,district,grand_total,drg,drg_grand_total,state_grand_total,school_code
0,None,Achievement First Hartford Academy Inc.,52769.50000,None,NaN,68786.73711,None
1,2790013,Amistad Academy District,56243.54667,None,NaN,68786.73711,2790013
2,0010011,Andover School District,59970.76923,C,64867.747545,68786.73711,0010011
3,0020011,Ansonia School District,55331.25000,H,70578.395200,68786.73711,0020011
4,2440014,Area Cooperative Educational Services,67315.08403,None,NaN,68786.73711,2440014
5,0030011,Ashford School District,56571.79487,E,63440.823205,68786.73711,0030011
6,0040011,Avon School District,80123.66197,B,71772.560280,68786.73711,0040011
7,0050011,Barkhamsted School District,67363.52174,C,64867.747545,68786.73711,0050011
8,0070011,Berlin School District,69066.94686,D,67826.620525,68786.73711,0070011
9,0080011,Bethany School District,55983.76316,C,64867.747545,68786.73711,0080011


In [60]:
combined.to_csv("../clean/for_db/combined_compensation_grand_total.csv",index=False)